# ReleasePipeline Tutorial

This notebook demonstrates how to use the **ReleasePipeline** agent for automated software releases.

## What It Does

The ReleasePipeline automates the entire release workflow:

1. **Validate Changelog** (LLM) - Validates your changelog format and generates a commit message
2. **Create Branch** (Shell) - Creates a release branch: `release/v{version}`
3. **Stage Changes** (Shell) - Stages all changes with `git add -A`
4. **Commit Changes** (Shell) - Commits with the AI-generated message
5. **Push Branch** (Shell) - Pushes the release branch to remote
6. **Create PR** (Shell) - Creates a pull request via GitHub CLI
7. **Generate Release Notes** (LLM) - Creates formatted release notes
8. **Generate Summary** (LLM) - Provides final status and next steps

## When to Use It

Use this pipeline when you want to:
- Automate repetitive release steps
- Ensure consistent commit messages and PR descriptions
- Generate professional release notes automatically
- Create a release branch and PR (not push directly to main)

## Setup

### Provider Configuration

The pipeline uses Claude to generate commit messages, PR descriptions, and release notes. You can use either provider:

**Option 1: Claude Code SDK** (development - flat $20/month)
```bash
export AGENT_WORKSHOP_ENV=development
export CLAUDE_SDK_ENABLED=true
```

**Option 2: Anthropic API** (production - pay-per-token)
```bash
export AGENT_WORKSHOP_ENV=production
export ANTHROPIC_API_KEY=sk-ant-...
```

In [ ]:
# Check current environment configuration
import os

env = os.getenv("AGENT_WORKSHOP_ENV", "development")
sdk_enabled = os.getenv("CLAUDE_SDK_ENABLED", "true").lower() == "true"
has_api_key = bool(os.getenv("ANTHROPIC_API_KEY"))

print(f"Environment: {env}")
print(f"Claude SDK Enabled: {sdk_enabled}")
print(f"Anthropic API Key: {'✓ Set' if has_api_key else '✗ Not set'}")

if env == "development" and sdk_enabled:
    print("\n→ Using Claude Code SDK (flat rate)")
elif has_api_key:
    print("\n→ Using Anthropic API (pay-per-token)")
else:
    print("\n⚠️ No provider configured. Set CLAUDE_SDK_ENABLED=true or ANTHROPIC_API_KEY")

In [ ]:
# Import the pipeline
from agent_workshop.agents.software_dev import ReleasePipeline
from agent_workshop import Config

# Create configuration (auto-detects provider based on environment)
config = Config()

# Instantiate the pipeline
pipeline = ReleasePipeline(config)

print(f"Provider: {pipeline.provider_name}")
print(f"Model: {pipeline.model_name}")

## Understanding the Workflow

Before running, let's understand what the pipeline will do:

```
┌─────────────────────┐
│ validate_changelog  │ ← LLM validates format, generates commit message
└─────────┬───────────┘
          ▼
┌─────────────────────┐
│   create_branch     │ ← git checkout -b release/v{version}
└─────────┬───────────┘
          ▼
┌─────────────────────┐
│   stage_changes     │ ← git add -A
└─────────┬───────────┘
          ▼
┌─────────────────────┐
│   commit_changes    │ ← git commit -m "{commit_message}"
└─────────┬───────────┘
          ▼
┌─────────────────────┐
│    push_branch      │ ← git push -u origin release/v{version}
└─────────┬───────────┘
          ▼
┌─────────────────────┐
│     create_pr       │ ← gh pr create --base main ...
└─────────┬───────────┘
          ▼
┌─────────────────────┐
│ generate_release    │ ← LLM creates formatted release notes
│      _notes         │
└─────────┬───────────┘
          ▼
┌─────────────────────┐
│  generate_summary   │ ← LLM summarizes workflow results
└─────────────────────┘
```

## Input Format

The pipeline requires four inputs:

| Field | Type | Description |
|-------|------|-------------|
| `version` | str | Version number (e.g., "0.3.0") |
| `release_type` | str | Type: "major", "minor", or "patch" |
| `changelog_content` | str | Changelog text for this release |
| `base_branch` | str | Branch to create PR against (e.g., "main") |

In [ ]:
# Prepare changelog content
changelog = """## [0.4.0] - 2025-01-15

### Added
- New tutorial notebooks for all agents
- Provider configuration documentation
- Interactive examples in Jupyter

### Changed
- Improved error messages for configuration issues
- Updated README with notebook links

### Fixed
- Shell command quoting for special characters
"""

# Prepare input
release_input = {
    "version": "0.4.0",
    "release_type": "minor",
    "changelog_content": changelog,
    "base_branch": "main",
}

print("Release Input:")
print(f"  Version: {release_input['version']}")
print(f"  Type: {release_input['release_type']}")
print(f"  Base Branch: {release_input['base_branch']}")
print(f"\nChangelog Preview:\n{changelog[:200]}...")

## Running the Pipeline

⚠️ **Warning**: Running this pipeline will:
1. Create a new git branch
2. Stage and commit all current changes
3. Push to your remote repository
4. Create a pull request on GitHub

Make sure you're in a git repository with pending changes before running!

In [ ]:
# Check git status before running
import subprocess

result = subprocess.run(["git", "status", "--short"], capture_output=True, text=True)
print("Current git status:")
print(result.stdout if result.stdout else "(no changes)")

# Check current branch
branch_result = subprocess.run(["git", "branch", "--show-current"], capture_output=True, text=True)
print(f"\nCurrent branch: {branch_result.stdout.strip()}")

In [ ]:
# OPTIONAL: Dry run - inspect what would happen without executing
# This shows the graph structure without running any commands

graph = pipeline.build_graph()
print("Pipeline Steps:")
for node in graph.nodes:
    if not node.startswith("__"):  # Skip internal nodes
        print(f"  → {node}")

In [ ]:
# Execute the pipeline (ONLY RUN IF YOU WANT TO CREATE A REAL RELEASE)
# Uncomment the lines below to run:

# result = await pipeline.run(release_input)
# print("Pipeline Result:")
# import json
# print(json.dumps(result, indent=2, default=str))

## Understanding the Output

The pipeline returns a dictionary with results from each step:

```python
{
    # LLM-generated content
    "changelog_validation": {
        "valid": True,
        "issues": [],
        "commit_message": "feat(release): v0.4.0 - Add tutorial notebooks",
        "pr_body": "## Release v0.4.0\n\n..."
    },
    "commit_message": "feat(release): v0.4.0 - Add tutorial notebooks",
    "pr_body": "...",
    "release_notes": {
        "title": "v0.4.0",
        "body": "...",
        "highlights": [...],
        "breaking_changes": []
    },
    
    # Shell command results
    "branch_success": True,
    "branch_output": "Switched to new branch 'release/v0.4.0'",
    "stage_success": True,
    "commit_success": True,
    "commit_output": "[release/v0.4.0 abc1234] feat(release): v0.4.0...",
    "push_success": True,
    "pr_success": True,
    "pr_output": "https://github.com/user/repo/pull/123",
    
    # Final summary
    "final_result": {
        "success": True,
        "version": "0.4.0",
        "pr_url": "https://github.com/user/repo/pull/123",
        "next_steps": [
            "Review and merge the PR",
            "Create GitHub Release with tag v0.4.0",
            "Run: uv build && twine upload dist/*"
        ],
        "summary": "Release v0.4.0 created successfully..."
    }
}
```

## Safety Features

The ReleasePipeline is designed with safety in mind:

1. **Branch-based workflow** - Creates a release branch instead of pushing directly to main
2. **PR for review** - Creates a pull request so changes can be reviewed before merging
3. **Shell quoting** - Uses `shlex.quote()` to safely handle special characters
4. **Timeout enforcement** - Each shell command has a timeout (default 60-120 seconds)
5. **Exit code checking** - Verifies each command succeeded before proceeding

## Prerequisites

Before running the pipeline, ensure:

1. **Git configured**: `git config user.name` and `git config user.email` are set
2. **GitHub CLI installed**: `gh` command available and authenticated (`gh auth login`)
3. **Remote configured**: `git remote -v` shows your GitHub repository
4. **Clean working state**: No uncommitted changes you don't want in the release
5. **On correct branch**: Usually `main` or `develop`

In [ ]:
# Check prerequisites
import subprocess

def check_command(cmd, description):
    try:
        result = subprocess.run(cmd, capture_output=True, text=True, timeout=5)
        return result.returncode == 0, result.stdout.strip()
    except Exception as e:
        return False, str(e)

print("Prerequisites Check:")
print("=" * 40)

# Git user
ok, output = check_command(["git", "config", "user.name"], "Git user name")
print(f"{'✓' if ok else '✗'} Git user: {output if ok else 'Not configured'}")

# Git remote
ok, output = check_command(["git", "remote", "get-url", "origin"], "Git remote")
print(f"{'✓' if ok else '✗'} Remote: {output if ok else 'Not configured'}")

# GitHub CLI
ok, output = check_command(["gh", "--version"], "GitHub CLI")
print(f"{'✓' if ok else '✗'} GitHub CLI: {'Installed' if ok else 'Not found'}")

# GitHub CLI auth
ok, output = check_command(["gh", "auth", "status"], "GitHub auth")
print(f"{'✓' if ok else '✗'} GitHub Auth: {'Authenticated' if ok else 'Not authenticated'}")

## Next Steps

After running the pipeline:

1. **Review the PR** - Check the generated PR on GitHub
2. **Merge when ready** - Use GitHub's merge button
3. **Create GitHub Release** - Go to Releases → Draft new release → Tag: v{version}
4. **Publish to PyPI** (if applicable):
   ```bash
   uv build
   twine upload dist/*
   ```

## Related Tutorials

- [00_getting_started.ipynb](./00_getting_started.ipynb) - Framework overview
- [03_code_reviewer.ipynb](./03_code_reviewer.ipynb) - Code review agent
- [04_pr_pipeline.ipynb](./04_pr_pipeline.ipynb) - PR review workflow
- [07_blueprint_system.ipynb](./07_blueprint_system.ipynb) - Creating custom agents